In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, adjusted_rand_score

# Загрузка данных
df = pd.read_csv("../data/prepared_data_for_clusterization.csv")

# Признаки (без 'views')
feature_columns = [
    'duration', 'frame rate',  'pixel_count',
    'title_len', 'uppercase_ratio', 'emoji_count', 'has_exclamation',
    'category_Howto & Style', 'category_Music', 'category_News & Politics',
    'category_Nonprofits & Activis', 'category_People & Blogs',
    'category_Pets & Animals', 'category_Science & Technology',
    'category_Shows', 'category_Sports', 'category_Travel & Events',
    'description', 'hashtags'
]

X = df[feature_columns].values


# Подбор K по силуэту (без графиков)
K_range = range(2, 11)
best_k = 2
best_sil = -1

print("Подбор K по силуэтному коэффициенту:")
for k in K_range:
    kmeans = KMeans(n_clusters=k, init='k-means++', n_init=10, random_state=42)
    labels = kmeans.fit_predict(X)
    sil = silhouette_score(X, labels)
    print(f"  K={k:2d} → Silhouette = {sil:.4f}")
    if sil > best_sil:
        best_sil = sil
        best_k = k

print(f"\nВыбрано K = {best_k} (макс. силуэт = {best_sil:.4f})")

# Обучение финальной модели
kmeans = KMeans(n_clusters=best_k, init='k-means++', n_init=10, random_state=42)
cluster_labels = kmeans.fit_predict(X)
df['cluster'] = cluster_labels

print("\n" + "="*50)
print("a. ЭКСПЕРТНАЯ ОЦЕНКА")
print("="*50)
# === 1. Распределение по кластерам ===
print("\nРаспределение объектов по кластерам:")
cluster_counts = pd.Series(cluster_labels).value_counts().sort_index()
for k, cnt in cluster_counts.items():
    print(f"  Кластер {k}: {cnt} видео ({cnt / len(df) * 100:.1f}%)")

# === 2. Средние значения по кластерам ===
print("\nСредние значения числовых признаков по кластерам:")
numeric = ['duration', 'pixel_count', 
           'title_len', 'uppercase_ratio', 'emoji_count', 'has_exclamation']

cluster_means = df.groupby('cluster')[numeric].mean().round(3)
print(cluster_means.to_string())

# === 3. Доминирующие категории в кластерах ===
print("\nДоля категорий в кластерах (топ-3 на кластер):")
cat_cols = [c for c in df.columns if c.startswith('category_')]
cat_means = df.groupby('cluster')[cat_cols].mean()

for k in range(best_k):
    print(f"\nКластер {k}:")
    top_cats = cat_means.loc[k].nlargest(3)
    for cat, prob in top_cats.items():
        cat_name = cat.replace('category_', '')
        print(f"  • {cat_name:<25} → {prob:.3f}")

print("\nЭКСПЕРТНАЯ ОЦЕНКА:")
if best_k == 2:
    # Извлекаем данные
    t0, t1 = cluster_means.loc[0, 'title_len'], cluster_means.loc[1, 'title_len']
    p0, p1 = cluster_means.loc[0, 'pixel_count'], cluster_means.loc[1, 'pixel_count']
    d0, d1 = cluster_means.loc[0, 'duration'], cluster_means.loc[1, 'duration']
    
    # Топ-категории
    top0 = cat_means.loc[0].nlargest(1).index[0].replace('category_', '')
    top1 = cat_means.loc[1].nlargest(1).index[0].replace('category_', '')
    top0_val = cat_means.loc[0].nlargest(1).values[0]
    top1_val = cat_means.loc[1].nlargest(1).values[0]

    
# === 5. b. СРАВНЕНИЕ С РЕАЛЬНЫМИ РАЗБИЕНИЯМИ ===
print("\n" + "="*50)
print("b. СРАВНЕНИЕ С РЕАЛЬНЫМИ РАЗБИЕНИЯМИ")
print("="*50)

# Восстановление истинной категории
df['true_category'] = df[cat_cols].idxmax(axis=1).str.replace('category_', '')
ari_cat = adjusted_rand_score(df['true_category'], df['cluster'])
print(f"ARI (кластеры vs категории): {ari_cat:.4f}")

# Сравнение с популярностью (если есть 'views')
if 'views' in df.columns:
    y_pop = (df['views'] > 200).astype(int)
    ari_pop = adjusted_rand_score(y_pop, df['cluster'])
    print(f"ARI (кластеры vs популярность, views>200): {ari_pop:.4f}")
else:
    print("Столбец 'views' отсутствует — сравнение с популярностью невозможно.")

print("\nИнтерпретация:")
if abs(ari_cat) < 0.1:
    print("  → Кластеры почти не связаны с категориями — выявлена новая, скрытая структура данных.")
elif ari_cat < 0.3:
    print("  → Слабая связь с категориями — кластеры отражают иные закономерности (например, стиль описания).")
else:
    print("  → Умеренная/сильная связь с категориями — кластеры соответствуют жанровому делению.")
    
    # === 6. c. ВИЗУАЛИЗАЦИЯ (тепловая карта) ===
print("\n" + "="*50)
print("c. ВИЗУАЛИЗАЦИЯ")
print("="*50)
print("Генерация диаграмм...")

plt.figure(figsize=(14, 5))

# 1. Топ-3 категории по кластерам (столбчатая диаграмма)
plt.subplot(1, 2, 1)
top_n = 3
clusters = list(range(best_k))
x = np.arange(len(clusters))
width = 0.25

colors = plt.cm.tab10.colors
for i in range(top_n):
    values = []
    labels = []
    for k in clusters:
        top_cats = cat_means.loc[k].nlargest(top_n)
        values.append(top_cats.iloc[i] if i < len(top_cats) else 0)
        if i == 0:
            labels.append(top_cats.index[i].replace('category_', ''))
    plt.bar(x + i*width, values, width, 
            label=f'Топ-{i+1}' + (f': {labels[0]}' if i == 0 else ''), 
            color=colors[i % len(colors)], alpha=0.8)

plt.xlabel('Кластер')
plt.ylabel('Доля видео')
plt.title('Топ-3 категории в кластерах')
plt.xticks(x + width, [f'Кл.{k}' for k in clusters])
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 2. Профиль кластеров: ключевые числовые признаки
plt.subplot(1, 2, 2)

# Выбираем 5 наиболее различающихся признаков
diffs = (cluster_means.iloc[1] - cluster_means.iloc[0]).abs().sort_values(ascending=False)
top_features = diffs.head(5).index.tolist()

x = np.arange(len(top_features))
width = 0.35

# Значения для кластеров
cluster0_vals = cluster_means.loc[0, top_features].values
cluster1_vals = cluster_means.loc[1, top_features].values

plt.bar(x - width/2, cluster0_vals, width, label='Кластер 0', 
        color='steelblue', alpha=0.8)
plt.bar(x + width/2, cluster1_vals, width, label='Кластер 1', 
        color='tomato', alpha=0.8)

# Добавляем точки для наглядности
plt.scatter(x - width/2, cluster0_vals, color='navy', s=50, zorder=5)
plt.scatter(x + width/2, cluster1_vals, color='darkred', s=50, zorder=5)

plt.xlabel('Признак')
plt.ylabel('Стандартизованное значение')
plt.title('Профиль кластеров: ключевые различия')
plt.xticks(x, [f.split('_')[-1] if '_' in f else f for f in top_features], 
           rotation=30, ha='right')
plt.axhline(0, color='gray', linestyle='--', linewidth=0.8, alpha=0.7)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
# Сохраняем фигуру в файл
plt.savefig("cluster_profiles.png", dpi=300, bbox_inches='tight')

plt.show()

